# Лабораторная работа №3. Моделирование непрерывных случайных величин (НСВ)

## Ширяева Ольга, 4 курс, 3 группа

## Вариант 12

# Задание 1.

Осуществить моделирование n = 10000 реализаций случайной величины из нормального закона распределения N(m, s2) с заданными параметрами. Для моделирования воспользоваться алгоритмом, основанным на ЦПТ; (в качестве количества используемых слагаемых можно взять N = 48, или 192, но должна быть возможность быстро изменить данный параметр). Вычислить несмещенные оценки математического ожидания и дисперсии, сравнить их с истинными значениями.

### m = -1, s2 = 4

In [385]:
n = 10000
m = -1
s = 2

In [386]:
from random import random
N = 48
def get_next_standard_gauss():
    return (sum([random() for _ in range(N)])  - N/2) / (N ** 0.5 * (1 / 12) ** 0.5)


def get_next_gauss(m, s):
    return m + s * get_next_standard_gauss()


def gauss(m, s, n):
    for _ in range(n):
        yield get_next_gauss(m, s)

In [387]:
gauss_seq = list(gauss(m, s, n))

for i in range(10):
    print(gauss_seq[i])

-3.84064150842794
1.2427344752742937
-0.7283595276257486
-0.6788716300882989
-1.4944079352590478
2.1770622430987108
-3.977808775454893
-1.2903039595785692
1.6733223188519486
-0.4911544165568137


In [388]:
def expectation(seq):
    sum = 0.
    for value in seq:
        sum += value
        
    return sum / len(seq)


def variance(seq, exp):
    var = 0.
    for value in seq:
        var += (value - exp) ** 2
    
    return var / len(seq)

In [389]:
gauss_exp = expectation(gauss_seq)

In [390]:
print('Gaussian')
print('Theoretic : E = {}, D = {}'.format(m, s ** 2))
print('Actual :    E = {}, D = {}'.format(gauss_exp, variance(gauss_seq, gauss_exp)))

Gaussian
Theoretic : E = -1, D = 4
Actual :    E = -1.000228423961997, D = 4.017820813518201


# Задание 2.

Смоделировать n = 10000 случайных величин из заданных абсолютно непрерывных распределений. Вычислить несмещенные оценки математического ожидания и дисперсии, сравнить их с истинными значениями (если это возможно). Если математического ожидания не существует, то вычислить выборочное значение медианы и сравнить его с теоретическим.

### 12) Коши C(a,b), a = -1, b = 1, Логистическое LG(a,b), a = 2, b = 3


### Коши

#### http://help.prognoz.com/ru/mergedProjects/Lib/05_statistics/distribution/lib_cauchydistribution.htm
### https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9A%D0%BE%D1%88%D0%B8


### Логистическое

#### https://ru.wikipedia.org/wiki/%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5

#### http://help.prognoz.com/ru/mergedProjects/Lib/05_statistics/distribution/lib_logisticdistribution.htm

In [391]:
a_cauchy = -1
b_cauchy = 1

a_log = 2
b_log = 3

In [392]:
from math import tan, pi, atan, exp, erf, sqrt

def cauchy(a, b, n):
    for i in range(n):
        yield a + b * tan (pi * (random() - 0.5))

In [393]:
cauchy_seq = list(cauchy(a_cauchy, b_cauchy, n))

for i in range(10):
    print(cauchy_seq[i])

-0.7987291133001733
4.664583278330684
-8.999324460168982
-1.9796211023583372
-4.81562278256261
-1.43741681360927
-0.5809401686038005
-0.6053330242116257
-4.200960282671815
4.891494086213687


In [394]:
sorted_seq = sorted(cauchy_seq)
median = (sorted_seq[int(n / 2) - 1] + sorted_seq[int(n / 2)]) / 2

print('Cauchy')
print('Median : Theory : {}, Actual : {}'.format(a_cauchy, median))

Cauchy
Median : Theory : -1, Actual : -0.9883896255551141


In [395]:
from numpy import log

def logistic(a, b, n):
    for i in range(n):
        x = random()
        yield a + b * log(x / (1 - x))

In [396]:
logistic_seq = list(logistic(a_log, b_log, n))

for i in range(10):
    print(logistic_seq[i])

-3.2726367873174986
3.3308586443248296
2.229116653177538
-1.7786693417466806
0.607818426650703
-5.759586677379458
-0.16568020448603882
1.718270735232413
3.8555251058266498
8.056719708252226


In [397]:
log_exp = expectation(logistic_seq)

In [398]:
print('Logistic')
print('Theoretic : E = {}, D = {}'.format(a_log, pi ** 2 * b_log ** 2 / 3))
print('Actual :    E = {}, D = {}'.format(log_exp, variance(logistic_seq, log_exp)))

Logistic
Theoretic : E = 2, D = 29.608813203268074
Actual :    E = 2.017361894495083, D = 29.401747268462994


In [399]:
def box_muller_normal(n):
    for i in range(int(n / 2)):
        x = random() * 2 - 1
        y = random() * 2 - 1
        s = x ** 2 + y ** 2
    
        while s > 1 or s == 0:
            x = random() * 2 - 1
            y = random() * 2 - 1
            s = x ** 2 + y ** 2
    
        yield x *( -2 * log(s) / s) ** 0.5
        yield y *( -2 * log(s) / s) ** 0.5

In [400]:
normal_seq = list(box_muller_normal(n))

for i in range(20):
    print(normal_seq[i])

0.6810822449063294
-0.2861250712762326
-0.9583866549414444
-1.5663070467585003
-0.7529955753456572
-1.7243390366296185
1.3633193767137166
-0.7730832546072045
-0.8220412013017753
0.507430662562422
-0.30663354978618423
1.1846918066017151
-0.7112209137558705
-0.23614916653032705
-1.7231596779365024
-0.5288743450242509
1.3428443943146913
-0.32236154017469665
-1.02285487480021
0.49370320084094976


In [401]:
def correlation(seq1, seq2):
    exp1 = expectation(seq1)
    exp2 = expectation(seq2)
    var1 = variance(seq1, exp1)
    var2 = variance(seq2, exp2)
    
    corr = 0.
    for i in range(len(seq1)):
        corr += (seq1[i] - exp1) * (seq2[i] - exp2)

    return corr / (var1 * var2) ** 0.5 / len(seq1)

In [402]:
seq1 = [normal_seq[i] for i in range(0, len(normal_seq), 2)]
seq2 = [normal_seq[i] for i in range(1, len(normal_seq), 2)]


corr = correlation(seq1, seq2)

print('Correlation between even and odd values : {}'.format(corr))

Correlation between even and odd values : -0.009938355477888352


In [403]:
def sort_and_count(n, rand_seq, prob_function):
    seq = sorted(rand_seq, key=float)
    size = len(seq)
    segment_size = int(size / n)
    probabilities = []
    frequencies = []
    j = 0
    for i in range(n):
        count = 0
        prob_x0 = prob_function(seq[j])
        while j < size and seq[j] < (i + 1) / n:
                j += 1
                count += 1
        frequencies.append(count)
        probabilities.append(prob_function(seq[j]) - prob_x0)
    return frequencies, segment_size, probabilities


def chi_square(frequencies, size, probabilities, delta):
    chi = 0
    for i , frequency in enumerate(frequencies):
        chi += (frequency - probabilities[i] * size) ** 2 / (probabilities[i] * size)
    print('chi-square : {}, delta : {}'.format(chi, delta))
    
    return chi < delta

In [404]:
def gauss_distribution(x):
    return 0.5 * (1 + erf((x - m) / (sqrt(2) * s)))

def cauchy_distribution(x):
    return 1 / pi * atan((x - a_cauchy) / b_cauchy) + 1 / 2

def logistic_distribution(x):
    return 1 / (1 + exp(-1 * (x - a_log) / b_log))

### Критерий хи-квадрат

In [405]:
delta_10_005 = 16.919
delta_40_005 = 54.572
epsilon = 0.05
k = 10

frequencies, segment_size, probabilities = sort_and_count(k, gauss_seq, gauss_distribution)

print('Gaussian chi-square')

chi_square(frequencies, len(gauss_seq), probabilities, delta_10_005)

Gaussian chi-square
chi-square : 14.173534008026085, delta : 16.919


True

In [406]:
frequencies, segment_size, probabilities = sort_and_count(k, cauchy_seq, cauchy_distribution)

print('Cauchy chi-square')

chi_square(frequencies, len(cauchy_seq), probabilities, delta_10_005)

Cauchy chi-square
chi-square : 9.7860549266372, delta : 16.919


True

In [407]:
frequencies, segment_size, probabilities = sort_and_count(k, logistic_seq, logistic_distribution)

print('Logistic chi-square')

chi_square(frequencies, len(logistic_seq), probabilities, delta_10_005)

Logistic chi-square
chi-square : 5.577370149770179, delta : 16.919


True

### Критерий Колмогорова

In [408]:
import numpy as np

def kolmogorov(seq):
    delta = 1.358
    sorted_seq = sorted(seq, key=float)
    n = len(sorted_seq)
    test_seq = np.array([float(i + 1) / n for i in range(n)])
    max_diff = max(list(map(abs, test_seq - sorted_seq)))
    value = sqrt(n) * max_diff
    
    print('kolmogorov : {}, delta : {}'.format(value, delta))
    
    return value < delta

In [409]:
cauchy_model_dist = [cauchy_distribution(i) for i in sorted(cauchy_seq)]
gauss_model_dist = [gauss_distribution(i) for i in sorted(gauss_seq)]
logistic_model_dist = [logistic_distribution(i) for i in sorted(logistic_seq)]

In [410]:
print('kolmogorov')
print()
print('Cauchy : {}'.format(kolmogorov(cauchy_model_dist)))
print('Gaussian : {}'.format(kolmogorov(gauss_model_dist)))
print('Logistic : {}'.format(kolmogorov(logistic_model_dist)))

kolmogorov

kolmogorov : 0.5486059822043421, delta : 1.358
Cauchy : True
kolmogorov : 0.7212301638355456, delta : 1.358
Gaussian : True
kolmogorov : 0.4383872476881462, delta : 1.358
Logistic : True


In [411]:
def w_square(seq, delta):
    sorted_seq = sorted(seq, key=float)
    n = len(sorted_seq)
    test_seq = np.array([float(2 * i - 1) / (2 * n) for i in range(n)])
    nw2 = 1 / (12 * n)
    
    nw2 += sum([(seq[i] - test_seq[i]) ** 2 for i in range(n)])
    
    print('kolmogorov : {}, delta : {}'.format(nw2, delta))
    
    return nw2 < delta

### Критерий омега-квадрат

https://arhiuch.ru/lab10.html

In [412]:
delta_nw2_normal = 0.126
delta_nw2 = 0.461

print('W-square')
print()
print('Cauchy : {}'.format(w_square(cauchy_model_dist, delta_nw2)))
print('Gaussian : {}'.format(w_square(gauss_model_dist, delta_nw2_normal)))
print('Logistic : {}'.format(w_square(logistic_model_dist, delta_nw2)))

W-square

kolmogorov : 0.07258334477688765, delta : 0.461
Cauchy : True
kolmogorov : 0.08745800317576047, delta : 0.126
Gaussian : True
kolmogorov : 0.038530436922110424, delta : 0.461
Logistic : True
